# Testing Grounds
This notebook tests the performance of a series of bots from draftsimtools.

In [57]:
import numpy as np
import pandas as pd
from operator import itemgetter
from copy import deepcopy
import json

from sklearn.model_selection import train_test_split

import draftsimtools as ds

### Load M19 drafts. 
Data folder is downloadable from draft-data-files on slack

In [58]:
raw_drafts = ds.load_drafts("../data/subset3000/test.csv")
# raw_drafts = ds.load_drafts("../../data/m19_2.csv") # Previous import.

This cell isn't runnable without AllSets.json

In [59]:
# Here other folks load card lists, but I grab them from json instead
# m19_set = ds.create_set("data/m19_rating.tsv", "data/m19_land_rating.tsv")

with open('../data/AllSets.json', 'r',encoding='utf-8') as json_data:
    mtgJSON = json.load(json_data)
jsonSubset = mtgJSON['M19']['cards']
thisSet = {card['name'] : card for card in jsonSubset}

In [60]:
# Another (fancier) way to create a list of names + lots of other useful stuff
nameList = pd.DataFrame.from_dict(thisSet, orient='index') #, columns=['colors','rarity','type','convertedManaCost'])
nameList['Name'] = nameList.index                 # We need names as a column, not an index
nameList['index'] = range(len(nameList))
nameList = nameList.set_index('index')     # And we need a normal numerical index
nameList[1:5]

# Process names, then handle weird card names (those with commas)
nameList['Name'] = nameList.Name.str.replace(' ', '_')

# This utility method searches for "Name" column in nameList that have commas
nameList, raw_drafts = ds.fix_commas(nameList, raw_drafts) # Returns a tuple, as it updates both

# Prints namelist
nameList[1:5]

,artist,borderColor,colorIdentity,colors,convertedManaCost,edhrecRank,flavorText,foreignData,frameVersion,hasFoil,...,names,hasLeadershipSkills,loyalty,frameEffect,variations,isPromo,colorIndicator,faceConvertedManaCost,side,Name
index,,,,,,,,,,,,,,,,,,,,,
1,Eric Deschamps,black,[R],[R],3.0,2758.0,She learned a tragic lesson that day: even the...,[{'flavorText': 'An jenem Tag lernte sie eine ...,2015,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Act_of_Treason
2,Anthony Palumbo,black,[W],[W],2.0,9845.0,Inner strength is never seen until it makes al...,[{'flavorText': 'Innere Stärke bleibt verborge...,2015,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aegis_of_the_Heavens
3,Zoltan Boros,black,"[U, W]","[U, W]",4.0,15888.0,The best of their trade know every bolt of the...,[{'flavorText': 'Die Besten ihres Handwerks ke...,2015,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aerial_Engineer
4,Lucas Graciano,black,[U],[U],2.0,5517.0,"If you can't find a doorway, make one.",[{'flavorText': 'Wenn du keinen Eingang finden...,2015,True,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Aether_Tunnel


Load sets normally instead.

In [61]:
m19_set = ds.create_set("../data/m19_rating.tsv", "../data/m19_land_rating.tsv")

In [62]:
m19_set = m19_set.sort_values("Name").reset_index(drop=True)

In [63]:
# Process the drafts, deconstructing packs (hands) at every turn of every draft
drafts = ds.process_drafts(raw_drafts)

Processing draft: 0.


In [64]:
# Splits into (toy) training and test sets. NOTE: For real training, use all drafts.
subset_drafts = drafts[:20] # We are only looking at 1000 with this import.
#subset_drafts = drafts[5000:5500]
#train, test = train_test_split(subset_drafts, test_size = 0.4)

In [65]:
# This should simply add basic lands. 

# Make sure all cards are listed in the nameList; update if necessary
for iDraft in range(len(subset_drafts)):
    draft = subset_drafts[iDraft]
    for pack in draft:     
        for cardName in pack:
            try:
                pos = nameList[nameList.Name==cardName].index[0]
            except:
                print("---Unrecognized card: ",cardName) # All unrecognized cards here seem to be foil lands
                #  	colors 	rarity 	type 	convertedManaCost 	Name
                nameList = nameList.append({'colors':[],'rarity':'weird','type':'weird',
                                            'convertedManaCost':0,'Name':cardName},ignore_index=True)

---Unrecognized card:  Island_2
---Unrecognized card:  Plains_4
---Unrecognized card:  Plains_3
---Unrecognized card:  Swamp_4
---Unrecognized card:  Forest_3
---Unrecognized card:  Forest_2
---Unrecognized card:  Swamp_2
---Unrecognized card:  Island_3
---Unrecognized card:  Mountain_4
---Unrecognized card:  Mountain_3
---Unrecognized card:  Island_4
---Unrecognized card:  Swamp_1
---Unrecognized card:  Mountain_2
---Unrecognized card:  Swamp_3
---Unrecognized card:  Forest_1
---Unrecognized card:  Forest_4
---Unrecognized card:  Island_1
---Unrecognized card:  Plains_2
---Unrecognized card:  Plains_1
---Unrecognized card:  Mountain_1


In [66]:
print(len(nameList))

320


In [67]:
# Splits data into training and testing sets.
# This doesn't matter for some bots, but for others it does,
# so we want to make sure we evaluate on the same testing
# set for every bot. 
train, test = train_test_split(subset_drafts, test_size = 0.4)

In [68]:
# Tests the random and the raredrafting bots against each other
bot1 = ds.RandomBot()
# bot2 = ds.RaredraftBot(nameList) # These bots are not runnable without allsets json.
# bot3 = ds.ClassicBot(nameList)
bot4 = ds.RandomBot()

In [69]:
tester = ds.BotTester(subset_drafts)

In [70]:
#tester.evaluate_bots([bot1, bot2, bot3], ["RandomBot", "RaredraftBot", "ClassicBot"])
tester.evaluate_bots([bot1, bot4], ["RandomBot", "MiscBot"])
tester.report_evaluations()
tester.write_evaluations()

draft_num    10.500000
pick_num     23.000000
RandomBot     0.225556
MiscBot       0.230000
dtype: float64
Wrote correct to: output_files/exact_correct.tsv
Wrote fuzzy_correct to: output_files/fuzzy_correct.tsv
Wrote rank_error to: output_files/rank_error.tsv
Wrote card_acc to: output_files/card_accuracies.tsv


In [36]:
import torch

In [39]:
#Torch imports.
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data.dataset import Dataset

#Implement NN.
class DraftNet(nn.Module):
    
    def __init__(self, set_tensor):
        """Placeholder NN. Currently does nothing.
        
        param ss: number of cards in set
        param set_tensor: Mxss set tensor describing the set
        """
        super(DraftNet, self).__init__()
        
        # Load set tensor.
        self.set_tensor = set_tensor
        self.set_tensor_tranpose = torch.transpose(set_tensor, 0, 1)
        self.M, self.ss = self.set_tensor.shape
        self.half_ss = self.ss / 2
        
        # Specify layer sizes. 
        size_in = self.ss + self.M
        #size_in = self.ss
        size1 = self.ss
        size2 = self.ss
        size3 = self.ss
        size4 = self.ss
        size5 = self.ss
        size6 = self.ss
        size7 = self.ss
        size8 = self.ss
        
        self.ns = 0.01
        
        self.bn = nn.BatchNorm1d(self.ss + self.M)
        
        self.linear1 = torch.nn.Linear(size_in, size1)
        self.bn1 = nn.BatchNorm1d(size1)
        self.relu1 = torch.nn.LeakyReLU(negative_slope = self.ns)
        self.dropout1 = nn.Dropout(0.5)
        
        self.linear2 = torch.nn.Linear(size1, size2)
        self.bn2 = nn.BatchNorm1d(size2)
        self.relu2 = torch.nn.LeakyReLU(negative_slope = self.ns)
        self.dropout2 = nn.Dropout(0.5)
        
        self.linear3 = torch.nn.Linear(size2, size3)
        self.bn3 = nn.BatchNorm1d(size3)
        self.relu3 = torch.nn.LeakyReLU(negative_slope = self.ns)
        self.dropout3 = nn.Dropout(0.5)
        
        self.linear4 = torch.nn.Linear(size3, size4)
        self.relu4 = torch.nn.LeakyReLU(negative_slope = self.ns)
        self.dropout4 = nn.Dropout(0.5)
        
        self.linear5 = torch.nn.Linear(size3, size5)
        self.relu5 = torch.nn.LeakyReLU(negative_slope = self.ns)
        self.dropout5 = nn.Dropout(0.5)
        
        self.linear6 = torch.nn.Linear(size3, size6)
        self.relu6 = torch.nn.LeakyReLU(negative_slope = self.ns)
        self.dropout6 = nn.Dropout(0.5)
        
        self.linear7 = torch.nn.Linear(size3, size7)
        self.relu7 = torch.nn.LeakyReLU(negative_slope = self.ns)
        self.dropout7 = nn.Dropout(0.5)
        
        self.linear8 = torch.nn.Linear(size3, size8)
        self.relu8 = torch.nn.LeakyReLU(negative_slope = self.ns)
        
        
        #self.sm = torch.nn.Softmax()
                
    def forward(self, x):
        
        collection = x[:, :self.ss]
        
        #collection = self.bn(collection)
        
        pack = x[:, self.ss:]
        
        # Get features from set tensor. 
        features = torch.mm(collection, self.set_tensor_tranpose)
        collection_and_features = torch.cat((collection, features), 1)
        
        collection_and_features = self.bn(collection_and_features)
        
        #y = self.linear1(collection_and_features)
        y = self.linear1(collection_and_features)
        y = self.bn1(y)
        y = self.relu1(y)
        y = self.dropout1(y)
        
        y = self.linear2(y)
        y = self.bn2(y)
        y = self.relu2(y)
        y = self.dropout2(y)
        
        y = self.linear3(y)
        y = self.bn3(y)
        y = self.relu3(y)
        y = self.dropout3(y)

        y = self.linear4(y)
        #y = self.relu4(y)
        #y = self.dropout4(y)
        
        #y = self.linear5(y)
        #y = self.relu5(y)
        #y = self.dropout5(y)
        
        #y = self.linear6(y)
        #y = self.relu6(y)
        #y = self.dropout6(y)
        
        #y = self.linear7(y)
        #y = self.relu7(y)
        #y = self.dropout7(y)
        
        #y = self.linear8(y)
        #y = self.relu8(y)
        
        y = y * pack # Enforce cards in pack only.
        
        return y

#Create NN.
#net = DraftNet(st).cuda()
#print(net)

In [45]:
class draftnet_bot(Bot):

    #Class CONSTANTS
    #None for base bots but yours should go here in ALL_CAPS

    def __init__(self, cur_set, le, draftnet_path="./bots_data/draftnet_aug_22_2019.pt"):
        """
        :param cur_set: DataFrame for current set (sorted alphabetically)
        :param le: Label encoder that transforms numbers <-> cardnames
        :param draftnet_path: path to draftnet parameter file (.pt)
        """
        self.le = le
        self.cur_set = cur_set

        self.num_correct = 0
        self.num_total = 0

        # Need to define draftnet architecture to use torch.load().
        self.net = torch.load(draftnet_path)

    def rank_pack(self, draft_frame):
        intermediate_rep = le.transform(draft_frame)

    def __get_ranking(self, draft_frame):
        pass

    def get_performance(self):
        """
        Return tuple of number of correct picks so far.
        """
        return self.num_total, self.num_correct

    def get_top_pick(self, pack_rank):
        """
        Returns the card name of the top-ranked pick within a pack.
        """
        intermediate_rep = []

        return max(pack_rank, key = pack_rank.get)

    def clear_history(self):
        """Reset class variables."""
        self.num_correct = 0
        self.num_total = 0

NameError: name 'Bot' is not defined

In [40]:
test_net = torch.load("./bots_data/draftnet_aug_22_2019.pt")

In [42]:
le.transform(["Abnormal_Endurance", "Act_of_Treason"])


NameError: name 'le' is not defined

In [41]:
test_net

DraftNet(
  (bn): BatchNorm1d(306, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (linear1): Linear(in_features=306, out_features=285, bias=True)
  (bn1): BatchNorm1d(285, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu1): LeakyReLU(negative_slope=0.01)
  (dropout1): Dropout(p=0.5)
  (linear2): Linear(in_features=285, out_features=285, bias=True)
  (bn2): BatchNorm1d(285, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu2): LeakyReLU(negative_slope=0.01)
  (dropout2): Dropout(p=0.5)
  (linear3): Linear(in_features=285, out_features=285, bias=True)
  (bn3): BatchNorm1d(285, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu3): LeakyReLU(negative_slope=0.01)
  (dropout3): Dropout(p=0.5)
  (linear4): Linear(in_features=285, out_features=285, bias=True)
  (relu4): LeakyReLU(negative_slope=0.01)
  (dropout4): Dropout(p=0.5)
  (linear5): Linear(in_features=285, out_features=285, bias=True)
  (relu5): LeakyR